In [1]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import cluster, metrics

#試取資料
file = open("../log/n3 (1).pickle", "rb")
data = pickle.load(file)
file.close()
scene_info = data['ml']['scene_info']
scene_command = data['ml']['command']

print(data)

{'record_format_version': 2, 'ml': {'scene_info': [{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (93, 395), 'platform': (75, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50, 80), (75, 80), (100, 80), (125, 80)], 'hard_bricks': [(35, 50), (60, 50), (85, 50), (110, 50), (135, 50), (25, 60), (150, 60), (35, 70), (135, 70), (25, 80), (150, 80), (35, 90), (60, 90), (85, 90), (110, 90), (135, 90)]}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (93, 395), 'platform': (75, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50, 80), (75, 80), (100, 80), (125, 80)], 'hard_bricks': [(35, 50), (60, 50), (85, 50), (110, 50), (135, 50), (25, 60), (150, 60), (35, 70), (135, 70), (25, 80), (150, 80), (35, 90), (60, 90), (85, 90), (110, 90), (135, 90)]}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (100, 388), 'platform': (70, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50

{'record_format_version': 2, 'ml': {'scene_info': [{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (93, 395), 'platform': (75, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50, 80), (75, 80), (100, 80), (125, 80)], 'hard_bricks': [(35, 50), (60, 50), (85, 50), (110, 50), (135, 50), (25, 60), (150, 60), (35, 70), (135, 70), (25, 80), (150, 80), (35, 90), (60, 90), (85, 90), (110, 90), (135, 90)]}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (93, 395), 'platform': (75, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50, 80), (75, 80), (100, 80), (125, 80)], 'hard_bricks': [(35, 50), (60, 50), (85, 50), (110, 50), (135, 50), (25, 60), (150, 60), (35, 70), (135, 70), (25, 80), (150, 80), (35, 90), (60, 90), (85, 90), (110, 90), (135, 90)]}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (100, 388), 'platform': (70, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50

{'record_format_version': 2, 'ml': {'scene_info': [{'frame': 0, 'status': 'GAME_ALIVE', 'ball': (93, 395), 'platform': (75, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50, 80), (75, 80), (100, 80), (125, 80)], 'hard_bricks': [(35, 50), (60, 50), (85, 50), (110, 50), (135, 50), (25, 60), (150, 60), (35, 70), (135, 70), (25, 80), (150, 80), (35, 90), (60, 90), (85, 90), (110, 90), (135, 90)]}, {'frame': 1, 'status': 'GAME_ALIVE', 'ball': (93, 395), 'platform': (75, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50, 80), (75, 80), (100, 80), (125, 80)], 'hard_bricks': [(35, 50), (60, 50), (85, 50), (110, 50), (135, 50), (25, 60), (150, 60), (35, 70), (135, 70), (25, 80), (150, 80), (35, 90), (60, 90), (85, 90), (110, 90), (135, 90)]}, {'frame': 2, 'status': 'GAME_ALIVE', 'ball': (100, 388), 'platform': (70, 400), 'bricks': [(50, 60), (75, 60), (100, 60), (125, 60), (60, 70), (85, 70), (110, 70), (50

In [2]:
for i in range(2, 14):
    path = "../log/n3 (" + str(i) + ").pickle"
    file = open(path, "rb")
    data = pickle.load(file)
    scene_info = scene_info + data['ml']['scene_info']
    scene_command = scene_command + data['ml']['command']
    file.close()
    
print(len(scene_info))
print(len(scene_command))

28908
28908
28908
28908
28908
28908


In [3]:
# 提取特徵
Ball_x = []
Ball_y = []
Platform = []
command = []

for i, s in enumerate(scene_info[1:-2]):
    Ball_x.append(s['ball'][0])
    Ball_y.append(s['ball'][1])
    Platform.append(s['platform'][0])
            
for c in scene_command[1:-2]:
    if c == "MOVE_LEFT":
        command.append(1)
    elif c == "MOVE_RIGHT":
        command.append(2)
    else:
        command.append(0)


numpy_data = np.array([Ball_x, Ball_y, Platform])
feature = np.transpose(numpy_data) 
answer = np.transpose(command)

print(feature)
print(feature.shape)
print(answer)
print(answer.shape)

[[ 93 395  75]
 [100 388  70]
 [107 381  65]
 ...
 [ 70 115  25]
 [ 80 108  20]
 [ 90 101  20]]
(28905, 3)
[1 1 2 ... 1 0 2]
(28905,)
[[ 93 395  75]
 [100 388  70]
 [107 381  65]
 ...
 [ 70 115  25]
 [ 80 108  20]
 [ 90 101  20]]
(28905, 3)
[1 1 2 ... 1 0 2]
(28905,)
[[ 93 395  75]
 [100 388  70]
 [107 381  65]
 ...
 [ 70 115  25]
 [ 80 108  20]
 [ 90 101  20]]
(28905, 3)
[1 1 2 ... 1 0 2]
(28905,)


In [4]:
#資料劃分
x_train, x_test, y_train, y_test = train_test_split(feature, answer, test_size=0.3, random_state=9)
#參數區間
param_grid = {'n_neighbors':[1, 2, 3]}
#交叉驗證 
cv = StratifiedShuffleSplit(n_splits=3, test_size=0.3, random_state=9)
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv, verbose=10, n_jobs=-1) #n_jobs為平行運算的數量
grid.fit(x_train, y_train)
grid_predictions = grid.predict(x_test)

#儲存
file = open('model.pickle', 'wb')
pickle.dump(grid, file)
file.close()

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    5.8s remaining:   20.7s
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:    5.9s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    6.0s remaining:    7.5s
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed:    6.0s remaining:    4.8s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    6.0s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    5.8s remaining:   20.7s
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:    5.9s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    6.0s remaining:    7.5s
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed:    6.0s remaining:    4.8s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    6.0s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    5.8s remaining:   20.7s
[Parallel(n_jobs=-1)]: Done   3 o

In [5]:
#最佳參數
print(grid.best_params_)
#預測結果
#print(grid_predictions)
#混淆矩陣
print(confusion_matrix(y_test, grid_predictions))
#分類結果
print(classification_report(y_test, grid_predictions))

{'n_neighbors': 3}
[[ 728  523  575]
 [ 785 1964  807]
 [ 859  780 1651]]
              precision    recall  f1-score   support

           0       0.31      0.40      0.35      1826
           1       0.60      0.55      0.58      3556
           2       0.54      0.50      0.52      3290

    accuracy                           0.50      8672
   macro avg       0.48      0.48      0.48      8672
weighted avg       0.52      0.50      0.51      8672

{'n_neighbors': 3}
[[ 728  523  575]
 [ 785 1964  807]
 [ 859  780 1651]]
              precision    recall  f1-score   support

           0       0.31      0.40      0.35      1826
           1       0.60      0.55      0.58      3556
           2       0.54      0.50      0.52      3290

    accuracy                           0.50      8672
   macro avg       0.48      0.48      0.48      8672
weighted avg       0.52      0.50      0.51      8672

{'n_neighbors': 3}
[[ 728  523  575]
 [ 785 1964  807]
 [ 859  780 1651]]
              pr